<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import os
import datetime
import gc

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Edit data directory here
DATA_DIR = "./input/techjam"


def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

def get_prep_data(DATA_DIR):
    # Reading Files
    cc = pd.read_csv(os.path.join(DATA_DIR,'cc.csv'),parse_dates=['pos_dt'])
    demo = pd.read_csv(os.path.join(DATA_DIR,'demographics.csv'))
    kplus = pd.read_csv(os.path.join(DATA_DIR,'kplus.csv'),parse_dates=['sunday'])

    train = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))
    test = pd.read_csv(os.path.join(DATA_DIR,'test.csv'))

    # Set-up
    cc_mapper = demo[['id','cc_no']].copy()
    demo = demo.drop('cc_no',axis=1).drop_duplicates().reset_index(drop=True)
    label = pd.concat([train,test],axis=0,ignore_index=True)
    demo = demo.merge(label, on='id')
    demo['ocp_cd'] = demo['ocp_cd'].fillna(0).astype(int)
    demo.set_index('id',inplace=True)
    kplus.set_index('id',inplace=True)
    joined_cc = cc.merge(cc_mapper, on='cc_no', how='inner').drop('cc_no', axis=1)

    kplus['month'] = kplus['sunday'].dt.month
    kplus['month'] = 'month'+ kplus['month'].astype(str)
    joined_cc['month'] = joined_cc.pos_dt.dt.month
    joined_cc['month'] = 'month'+ joined_cc['month'].astype(str)

    bank_holidays = ['2018-01-01','2018-01-02','2018-03-01','2018-04-06','2018-04-13',
                    '2018-04-14','2018-04-15','2018-04-16','2018-05-01','2018-05-29']
    joined_cc['is_holiday'] = joined_cc['pos_dt'].isin([datetime.datetime.strptime(i, '%Y-%m-%d') for i in bank_holidays]).astype(int)
    joined_cc['is_weekend'] = joined_cc['pos_dt'].dt.weekday.isin([0,6]).astype(int)
    joined_cc['is_holiday'] = 'holiday'+joined_cc['is_holiday'].astype(str)
    joined_cc['is_weekend'] = 'weekend'+joined_cc['is_weekend'].astype(str)
    joined_cc['quarter'] = 'q'+((joined_cc['pos_dt'].dt.month>=4)+1).astype(str)


    # Adding log 
    demo['income'] = demo['income'].apply(np.log1p)

   


    # Target Encoding
    demo = demo.reset_index()
    demo['cc_cnt'] = demo['id'].map(cc_mapper.groupby('id').cc_no.count())
    demo['has_kp'] = demo['id'].isin(kplus.index).astype(int)

    # Crossing categorical features as another feature [374 / 336]
    demo['age_gnd'] = demo['gender'].astype(str)+demo['age'].astype(str)
    demo['gnd_ocp'] = demo['gender'].astype(str)+demo['ocp_cd'].astype(str)
    demo['age_ocp'] = demo['age'].astype(str)+demo['ocp_cd'].astype(str)

    # Left age out of categorical features since it's ordinal
    categorical_features = ['gender','ocp_cd','age_gnd','gnd_ocp','age_ocp']

    # Target Encoding, code modified from [374]
    for feature in categorical_features + ['age']:
        means_per_group = demo[demo['income']>0].groupby(feature)['income'].mean()
        demo[feature+'_mean'] = demo[feature].map(means_per_group)

        count_per_group = demo[demo['income']>0].groupby(feature)['income'].count()
        demo[feature+'_count'] = demo[feature].map(count_per_group)
    demo.set_index('id',inplace=True)
    
     # Preping Training data
    train = demo.copy()

    # Normal Total Groupby
    kplus_tot = kplus.groupby('id').agg({'kp_txn_count':'sum','kp_txn_amt':'sum'}).copy()
    kplus_mm_tot = kplus.groupby(['id','month']).agg({'kp_txn_count':'sum','kp_txn_amt':'sum'}).unstack(level=1).copy()
    kplus_mm_tot.columns = ['_'.join([str(c) for c in lst]) for lst in kplus_mm_tot.columns]

    # CreditCard Total Groupby
    cc_tot = joined_cc.groupby('id').agg({'cc_txn_amt':['count','sum']}).copy()
    cc_tot.columns = ['_'.join(i) for i in cc_tot.columns]

    # CreditCard Monthly Groupby
    combined_cc = pd.pivot_table(joined_cc, index= 'id', columns= 'month', values= 'cc_txn_amt', aggfunc= [np.mean, min, max, np.sum, 'count', np.var, percentile(10), percentile(90)])
    combined_cc.columns = ['cc_'+'_'.join([str(c) for c in lst]) for lst in combined_cc.columns]


    # CreditCard Pompus Features
    combined_cc_holiday = pd.pivot_table(joined_cc, index= 'id', columns= 'is_holiday', values= 'cc_txn_amt', aggfunc= [np.mean, min, max, np.sum, 'count', np.var, percentile(10), percentile(90)])
    combined_cc_weekend = pd.pivot_table(joined_cc, index= 'id', columns= 'is_weekend', values= 'cc_txn_amt', aggfunc= [np.mean, min, max, np.sum, 'count', np.var, percentile(10), percentile(90)])
    combined_cc_quarter = pd.pivot_table(joined_cc, index= 'id', columns= 'quarter', values= 'cc_txn_amt', aggfunc= [np.mean, min, max, np.sum, 'count', np.var, percentile(10), percentile(90)])
    combined_cc_holiday.columns = ['cc_'+'_'.join([str(c) for c in lst]) for lst in combined_cc_holiday.columns]
    combined_cc_weekend.columns = ['cc_'+'_'.join([str(c) for c in lst]) for lst in combined_cc_weekend.columns]
    combined_cc_quarter.columns = ['cc_'+'_'.join([str(c) for c in lst]) for lst in combined_cc_quarter.columns]

    # Joining all together
    train = train.join(kplus_tot).join(kplus_mm_tot).join(cc_tot).join(combined_cc).join(combined_cc_holiday).join(combined_cc_weekend).join(combined_cc_quarter).fillna(0)

    # Quick Normalization
#     for col in [ i for i in train.columns if i not in ['id','gender','ocp_cd','age','income']]:
#         train[col] = StandardScaler().fit_transform(train[col].values.reshape(-1, 1))
#         train[col] = train[col].fillna(0)
#         if 'amt' in col:
#             train[col] = train[col].apply(np.log1p)

    X_train = train[train['income']>0].drop('income',axis=1).copy()
    y_train = pd.DataFrame(train[train['income']>0]['income']).copy()
    X_test = train[train['income']<=0].drop('income',axis=1).copy()

    return X_train, y_train, X_test

In [2]:
X_train, y_train, X_test =  get_prep_data(DATA_DIR)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [3]:
cat_feature = ['gender','ocp_cd','age_gnd','gnd_ocp','age_ocp', 'age']

In [4]:
def techjam_score(y_pred, y_true):
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    
    return 100 - 100 * np.mean((y_pred-y_true) ** 2 / (np.minimum(2*y_true, y_pred) + y_true)**2)

In [5]:
def techjam_feval_log(y_pred, dtrain):
    y_true = dtrain.get_label()
    return 'techjam_score', techjam_score(np.exp(y_pred), np.exp(y_true)), True

In [8]:
for cat in cat_feature:
    X_test[cat] =X_test[cat].astype(int)
    X_train[cat] =X_train[cat].astype(int)

train_data = lightgbm.Dataset(X_train, label=y_train, categorical_feature=cat_feature , free_raw_data=False)

num_leaves_choices = [15, 31, 63, 127, 200, 255, 300, 350, 400,511 ,600]
ft_frac_choices = [0.6, 0.7, 0.8, 0.9, 1.0]
bagging_frac_choices = [0.6, 0.7, 0.8, 0.9, 1.0]

# We will store the cross validation results in a simple list,
# with tuples in the form of (hyperparam dict, cv score):
cv_results = []


for num_lv in tqdm_notebook(num_leaves_choices):
    for bg_fac in bagging_frac_choices:
        for ft_fac in ft_frac_choices:
            hyperparams = {"boosting_type":'gbdt',
                            "objective": 'mape',
                            "metrics": 'None',
                            "num_leaves": num_lv,
                            "feature_fraction": ft_fac,
                            "bagging_fraction": bg_fac,
                            "learning_rate": 0.01
                                     }
            validation_summary = lightgbm.cv(hyperparams,
                                            train_data,
                                            num_boost_round=10000,
                                            nfold=5,
                                            feval=techjam_feval_log,
                                            stratified=False,
                                            shuffle=True,
                                            early_stopping_rounds=50,
                                            verbose_eval=10)
            
            optimal_num_trees = len(validation_summary["techjam_score-mean"])
            
            # to the hyperparameter dictionary:
            hyperparams["num_boost_round"] = optimal_num_trees

           # And we append results to cv_results:
            cv_results.append((hyperparams, validation_summary["techjam_score-mean"][-1]))

C:\Users\KTHEONE\AppData\Roaming\Python\Python37\site-packages\lightgbm\basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[10]	cv_agg's techjam_score: 89.4646 + 0.0831847
[20]	cv_agg's techjam_score: 89.8475 + 0.0847172
[30]	cv_agg's techjam_score: 90.1744 + 0.0843779
[40]	cv_agg's techjam_score: 90.4504 + 0.0837397
[50]	cv_agg's techjam_score: 90.6894 + 0.0832409
[60]	cv_agg's techjam_score: 90.8945 + 0.08055
[70]	cv_agg's techjam_score: 91.0698 + 0.0790555
[80]	cv_agg's techjam_score: 91.2157 + 0.0792943
[90]	cv_agg's techjam_score: 91.3422 + 0.0803296
[100]	cv_agg's techjam_score: 91.4488 + 0.0803911
[110]	cv_agg's techjam_score: 91.537 + 0.0805745
[120]	cv_agg's techjam_score: 91.6149 + 0.0809748
[130]	cv_agg's techjam_score: 91.6818 + 0.0813358
[140]	cv_agg's techjam_score: 91.7386 + 0.0812439
[150]	cv_agg's techjam_score: 91.7867 + 0.0818654
[160]	cv_agg's techjam_score: 91.8276 + 0.0822483
[170]	cv_agg's techjam_score: 91.8632 + 0.0821245
[180]	cv_agg's techjam_score: 91.8926 + 0.0828241
[190]	cv_agg's techjam_score: 91.9171 + 0.0822635
[200]	cv_agg's techjam_score: 91.9369 + 0.0813582
[210]	cv_agg

[110]	cv_agg's techjam_score: 91.5409 + 0.0800151
[120]	cv_agg's techjam_score: 91.6167 + 0.0810063
[130]	cv_agg's techjam_score: 91.6819 + 0.0815286
[140]	cv_agg's techjam_score: 91.7376 + 0.0810022
[150]	cv_agg's techjam_score: 91.7849 + 0.0800811
[160]	cv_agg's techjam_score: 91.824 + 0.0805921
[170]	cv_agg's techjam_score: 91.8576 + 0.0807031
[180]	cv_agg's techjam_score: 91.8852 + 0.0802199
[190]	cv_agg's techjam_score: 91.909 + 0.0803653
[200]	cv_agg's techjam_score: 91.9285 + 0.0813981
[210]	cv_agg's techjam_score: 91.9455 + 0.0818003
[220]	cv_agg's techjam_score: 91.9575 + 0.0826789
[230]	cv_agg's techjam_score: 91.9697 + 0.0829923
[240]	cv_agg's techjam_score: 91.9789 + 0.0830233
[250]	cv_agg's techjam_score: 91.9854 + 0.0838525
[260]	cv_agg's techjam_score: 91.9903 + 0.084479
[270]	cv_agg's techjam_score: 91.9952 + 0.0838252
[280]	cv_agg's techjam_score: 92.0002 + 0.0836926
[290]	cv_agg's techjam_score: 92.0032 + 0.0843704
[300]	cv_agg's techjam_score: 92.0059 + 0.0843571
[31

[210]	cv_agg's techjam_score: 91.9404 + 0.0831749
[220]	cv_agg's techjam_score: 91.9531 + 0.0828227
[230]	cv_agg's techjam_score: 91.9648 + 0.0825812
[240]	cv_agg's techjam_score: 91.9738 + 0.0836375
[250]	cv_agg's techjam_score: 91.983 + 0.0836158
[260]	cv_agg's techjam_score: 91.99 + 0.0827741
[270]	cv_agg's techjam_score: 91.9954 + 0.0813545
[280]	cv_agg's techjam_score: 91.9999 + 0.0799753
[290]	cv_agg's techjam_score: 92.0037 + 0.0795931
[300]	cv_agg's techjam_score: 92.0064 + 0.0787076
[310]	cv_agg's techjam_score: 92.0072 + 0.0785226
[320]	cv_agg's techjam_score: 92.0068 + 0.0768624
[330]	cv_agg's techjam_score: 92.0058 + 0.0754882
[340]	cv_agg's techjam_score: 92.0051 + 0.0742236
[350]	cv_agg's techjam_score: 92.0026 + 0.0732868
[360]	cv_agg's techjam_score: 92.0006 + 0.0732087
[10]	cv_agg's techjam_score: 89.4712 + 0.0826981
[20]	cv_agg's techjam_score: 89.8585 + 0.08249
[30]	cv_agg's techjam_score: 90.1876 + 0.0840526
[40]	cv_agg's techjam_score: 90.4691 + 0.0848608
[50]	cv_a

[100]	cv_agg's techjam_score: 91.6492 + 0.0724437
[110]	cv_agg's techjam_score: 91.7377 + 0.0720838
[120]	cv_agg's techjam_score: 91.81 + 0.0709733
[130]	cv_agg's techjam_score: 91.8718 + 0.0700299
[140]	cv_agg's techjam_score: 91.9212 + 0.0694196
[150]	cv_agg's techjam_score: 91.9603 + 0.0695449
[160]	cv_agg's techjam_score: 91.9935 + 0.067342
[170]	cv_agg's techjam_score: 92.0197 + 0.0671566
[180]	cv_agg's techjam_score: 92.04 + 0.0668642
[190]	cv_agg's techjam_score: 92.0564 + 0.0662904
[200]	cv_agg's techjam_score: 92.0685 + 0.0674713
[210]	cv_agg's techjam_score: 92.0771 + 0.0673746
[220]	cv_agg's techjam_score: 92.0831 + 0.0666509
[230]	cv_agg's techjam_score: 92.0871 + 0.0657975
[240]	cv_agg's techjam_score: 92.0905 + 0.0653942
[250]	cv_agg's techjam_score: 92.0915 + 0.0657816
[260]	cv_agg's techjam_score: 92.0914 + 0.0655837
[270]	cv_agg's techjam_score: 92.0902 + 0.0667717
[280]	cv_agg's techjam_score: 92.0887 + 0.067157
[290]	cv_agg's techjam_score: 92.0863 + 0.0685302
[300]	

...

[70]	cv_agg's techjam_score: 91.5019 + 0.0775631
[80]	cv_agg's techjam_score: 91.6515 + 0.0775555
[90]	cv_agg's techjam_score: 91.7744 + 0.0796253
[100]	cv_agg's techjam_score: 91.8746 + 0.0809206
[110]	cv_agg's techjam_score: 91.9535 + 0.0831137
[120]	cv_agg's techjam_score: 92.0188 + 0.0856203
[130]	cv_agg's techjam_score: 92.0701 + 0.0855341
[140]	cv_agg's techjam_score: 92.1082 + 0.0863402
[150]	cv_agg's techjam_score: 92.1377 + 0.0867222
[160]	cv_agg's techjam_score: 92.159 + 0.0869771
[170]	cv_agg's techjam_score: 92.1762 + 0.0875478
[180]	cv_agg's techjam_score: 92.1868 + 0.0889633
[190]	cv_agg's techjam_score: 92.1934 + 0.0893387
[200]	cv_agg's techjam_score: 92.1961 + 0.0901323
[210]	cv_agg's techjam_score: 92.1981 + 0.0928639
[220]	cv_agg's techjam_score: 92.1967 + 0.0940706
[230]	cv_agg's techjam_score: 92.1963 + 0.0947565
[240]	cv_agg's techjam_score: 92.194 + 0.0960102
[250]	cv_agg's techjam_score: 92.1898 + 0.0973467
[260]	cv_agg's techjam_score: 92.1851 + 0.0969557
[10]	

[200]	cv_agg's techjam_score: 92.2295 + 0.0835239
[210]	cv_agg's techjam_score: 92.2333 + 0.0835653
[220]	cv_agg's techjam_score: 92.2337 + 0.0840873
[230]	cv_agg's techjam_score: 92.234 + 0.0846372
[240]	cv_agg's techjam_score: 92.2324 + 0.0846986
[250]	cv_agg's techjam_score: 92.2306 + 0.0846641
[260]	cv_agg's techjam_score: 92.2264 + 0.0839806
[270]	cv_agg's techjam_score: 92.2223 + 0.0833303
[280]	cv_agg's techjam_score: 92.2172 + 0.0837059
[10]	cv_agg's techjam_score: 89.6082 + 0.0852809
[20]	cv_agg's techjam_score: 90.0951 + 0.085629
[30]	cv_agg's techjam_score: 90.4948 + 0.085501
[40]	cv_agg's techjam_score: 90.8278 + 0.0858125
[50]	cv_agg's techjam_score: 91.0976 + 0.0849516
[60]	cv_agg's techjam_score: 91.3214 + 0.0829584
[70]	cv_agg's techjam_score: 91.5066 + 0.0823631
[80]	cv_agg's techjam_score: 91.6584 + 0.0822373
[90]	cv_agg's techjam_score: 91.7809 + 0.081613
[100]	cv_agg's techjam_score: 91.8823 + 0.0816223
[110]	cv_agg's techjam_score: 91.963 + 0.0825341
[120]	cv_agg's

[30]	cv_agg's techjam_score: 90.5033 + 0.0834467
[40]	cv_agg's techjam_score: 90.8335 + 0.082598
[50]	cv_agg's techjam_score: 91.1042 + 0.081922
[60]	cv_agg's techjam_score: 91.3263 + 0.0800313
[70]	cv_agg's techjam_score: 91.5095 + 0.0799204
[80]	cv_agg's techjam_score: 91.6587 + 0.0801577
[90]	cv_agg's techjam_score: 91.7804 + 0.0787946
[100]	cv_agg's techjam_score: 91.8804 + 0.0806531
[110]	cv_agg's techjam_score: 91.9603 + 0.0821652
[120]	cv_agg's techjam_score: 92.0228 + 0.0836061
[130]	cv_agg's techjam_score: 92.0735 + 0.0848294
[140]	cv_agg's techjam_score: 92.1133 + 0.0859128
[150]	cv_agg's techjam_score: 92.1444 + 0.0875665
[160]	cv_agg's techjam_score: 92.1674 + 0.0876164
[170]	cv_agg's techjam_score: 92.1839 + 0.0884296
[180]	cv_agg's techjam_score: 92.1953 + 0.0891104
[190]	cv_agg's techjam_score: 92.203 + 0.0901514
[200]	cv_agg's techjam_score: 92.2055 + 0.0922499
[210]	cv_agg's techjam_score: 92.2072 + 0.0928214
[220]	cv_agg's techjam_score: 92.2059 + 0.0951884
[230]	cv_a

[170]	cv_agg's techjam_score: 92.1955 + 0.0820316
[180]	cv_agg's techjam_score: 92.2087 + 0.0827388
[190]	cv_agg's techjam_score: 92.2159 + 0.0832703
[200]	cv_agg's techjam_score: 92.2202 + 0.082951
[210]	cv_agg's techjam_score: 92.2224 + 0.0817667
[220]	cv_agg's techjam_score: 92.2217 + 0.0802416
[230]	cv_agg's techjam_score: 92.2224 + 0.0814707
[240]	cv_agg's techjam_score: 92.2193 + 0.081089
[250]	cv_agg's techjam_score: 92.2163 + 0.0805918
[260]	cv_agg's techjam_score: 92.2116 + 0.0801569
[10]	cv_agg's techjam_score: 89.6092 + 0.0849049
[20]	cv_agg's techjam_score: 90.0972 + 0.0864859
[30]	cv_agg's techjam_score: 90.498 + 0.0840457
[40]	cv_agg's techjam_score: 90.8315 + 0.0828738
[50]	cv_agg's techjam_score: 91.1022 + 0.0816469
[60]	cv_agg's techjam_score: 91.3273 + 0.0796636
[70]	cv_agg's techjam_score: 91.5114 + 0.0783795
[80]	cv_agg's techjam_score: 91.661 + 0.0787165
[90]	cv_agg's techjam_score: 91.786 + 0.0790505
[100]	cv_agg's techjam_score: 91.8872 + 0.07914
[110]	cv_agg's t

[60]	cv_agg's techjam_score: 91.3277 + 0.084608
[70]	cv_agg's techjam_score: 91.5101 + 0.0843112
[80]	cv_agg's techjam_score: 91.6612 + 0.0855333
[90]	cv_agg's techjam_score: 91.783 + 0.085263
[100]	cv_agg's techjam_score: 91.8829 + 0.0855091
[110]	cv_agg's techjam_score: 91.9619 + 0.0865071
[120]	cv_agg's techjam_score: 92.0253 + 0.0868351
[130]	cv_agg's techjam_score: 92.0754 + 0.0880094
[140]	cv_agg's techjam_score: 92.1142 + 0.0894132
[150]	cv_agg's techjam_score: 92.1443 + 0.0899092
[160]	cv_agg's techjam_score: 92.1655 + 0.0911334
[170]	cv_agg's techjam_score: 92.1821 + 0.0911208
[180]	cv_agg's techjam_score: 92.1925 + 0.0902871
[190]	cv_agg's techjam_score: 92.199 + 0.0901194
[200]	cv_agg's techjam_score: 92.2031 + 0.0910534
[210]	cv_agg's techjam_score: 92.2041 + 0.0911662
[220]	cv_agg's techjam_score: 92.2042 + 0.0923964
[230]	cv_agg's techjam_score: 92.2035 + 0.0917565
[240]	cv_agg's techjam_score: 92.2001 + 0.0940348
[250]	cv_agg's techjam_score: 92.1963 + 0.0938131
[260]	cv

In [9]:
sort_cv_result = sorted(cv_results, key=lambda tup:tup[1])

In [11]:
sort_cv_result[-1]

({'boosting_type': 'gbdt',
  'objective': 'mape',
  'metrics': 'None',
  'num_leaves': 511,
  'feature_fraction': 0.6,
  'bagging_fraction': 1.0,
  'learning_rate': 0.01,
  'num_boost_round': 234},
 92.23441356231982)

In [ ]:
#select parameter score > 92.21

In [14]:
### select best 10 models
MODELS=[] 
for params_and_score in tqdm_notebook(sort_cv_result[-10:]):
    params = params_and_score[0]
    model = lightgbm.train(params,
                train_data,
               )
    MODELS.append(model)

C:\Users\KTHEONE\AppData\Roaming\Python\Python37\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [15]:
### ensemble 10 models 
pred = []
for model in MODELS:
    y_pred = model.predict(X_test)
    y_pred = np.exp(y_pred)
    pred.append(y_pred)

In [16]:
pred=np.array(pred)
# perform ensemble
final_pred = pred.mean(axis=0)


In [28]:
### Create submission dataframe
submission = pd.DataFrame()
submission['id'] = [i for i in range(50001,65001)] 
submission['final_pred'] = final_pred